# IMPORTS

In [67]:
import torch
from torch import nn
from numpy import genfromtxt
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn.modules.activation import Softplus
import pandas as pd
from sklearn import train_test_split

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# HYPERPARAMETERS

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 1
hidden_size = 500
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# GET DATA

In [77]:
colnames = ['q', 'q_dot', 'label']
# dataset = genfromtxt('/content/drive/MyDrive/Robotics_2/1R_rigid_vertical/dataset.txt')
dataset = pd.read_csv('/content/drive/MyDrive/Robotics_2/1R_rigid_vertical/dataset.csv', names=colnames)

## divide the dataset into the  train and test

In [123]:
from torch.utils.data.dataset import TensorDataset
# test_dataset = dataset[0:len(dataset)//3]
# train_dataset = dataset[len(dataset)//3:len(dataset)]

# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

X_all = dataset.drop('label', axis=1).copy()
y_all = dataset['label'].copy()

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.25)

# X_train = torch.Tensor(X_train).float().to(device)
# y_train = torch.Tensor(y_train).float().to(device)
# X_test = torch.Tensor(X_test).float().to(device)
# y_test = torch.Tensor(y_test).float().to(device)

dataloader_train = DataLoader((X_train, y_train), batch_size=batch_size)
dataloader_test = DataLoader((X_test, y_test), batch_size=batch_size)

## see the dataset lengths

In [82]:
# print(len(dataset))
# print(len(train_dataset))
# print(len(test_dataset))
# print(len(train_loader))
# print(len(test_loader))
# # print(list(train_loader))

# print(dataset)
# print(X_all)
# print(y_all)

0          label
1        -2.2557
2        -6.2667
3         5.6835
4         5.5323
          ...   
49996     5.8255
49997    -5.7833
49998     5.0609
49999    -6.5111
50000     6.4514
Name: label, Length: 50001, dtype: object


# LNN

In [37]:
class LNN(nn.Module):
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.Softplus(),
        nn.Linear(input_size, hidden_size),
        nn.Softplus(),
        nn.Linear(input_size, hidden_size),
        nn.Softplus(),
        nn.Linear(hidden_size, 1)
    )

  def forward(self, x):
    return self.model(x)

## instance of the NN and optimizer

In [48]:
lnn = LNN(input_size, hidden_size).to(device)
opt = Adam(lnn.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()

# TRAIN

In [125]:
# n_total_steps = len(dataloader_train)
# for epoch in range(num_epochs):
#   for i, (q, q_dot, labels) in enumerate(dataloader_train):
#     labels = labels.to(device)
#     outputs = lnn(q, q_dot)
#     loss = loss_fn(outputs, labels)

#     loss.backward()
#     opt.step()
#     opt.zero_grad()
lnn.train()

LNN(
  (model): Sequential(
    (0): Linear(in_features=1, out_features=500, bias=True)
    (1): Softplus(beta=1, threshold=20)
    (2): Linear(in_features=1, out_features=500, bias=True)
    (3): Softplus(beta=1, threshold=20)
    (4): Linear(in_features=1, out_features=500, bias=True)
    (5): Softplus(beta=1, threshold=20)
    (6): Linear(in_features=500, out_features=1, bias=True)
  )
)